# DATA TRANSFORMATION - 16_Disnea
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [22]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'Data/'

## 1. Load

In [23]:
modulo = 'Disnea'

In [24]:
dcc = gf.diccionario_llaves()

In [25]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,fecha,Examen,year,month,year_month
0,1998401,2016-06-02,ESCALA DE DISNEA mMRC EPOC 0,2016,6,201606
1,1491934,2016-06-10,ESCALA DE DISNEA mMRC EPOC 0,2016,6,201606
2,801004,2019-07-12,ESCALA DE DISNEA mMRC EPOC 0,2019,7,201907
3,1226479,2019-07-26,ESCALA DE DISNEA mMRC EPOC 0,2019,7,201907
4,1241688,2016-09-01,ESCALA DE DISNEA mMRC EPOC 0,2016,9,201609


## 2. Transform

In [26]:
base.rename(columns={ 'Examen':'exam'}, inplace=True)

# Elimina dplicados
base = base.drop_duplicates()

# Elimina columnas sobran
base = base.drop(columns=['fecha','year_month'])

base['exam'] = base['exam'].apply( lambda x : str(x)[-1].lower() )

In [27]:
base.head()

,id,exam,year,month
0,1998401,0,2016,6
1,1491934,0,2016,6
2,801004,0,2019,7
3,1226479,0,2019,7
4,1241688,0,2016,9


* Number of delivered doses per month
* Number of different diseases per month

In [28]:
base_ = base.groupby(['id','year','month','exam'])['id'].count().\
    reset_index( name = 'epoc' )
base_.head()

,id,year,month,exam,epoc
0,524006,2015,9,4,1
1,524006,2016,10,3,1
2,524006,2017,1,1,1
3,524006,2017,5,1,1
4,524006,2017,8,1,1


In [31]:
# pivot table to have separate variables, nans replaced with zero
base_p = base_.pivot_table(index=['id','year','month'], columns='exam', values=['epoc']).reset_index()
base_p.columns = ['_'.join(col).strip() for col in base_p.columns.values]
base_p.fillna( 0 , inplace = True)

base_p.rename(columns={ 'id_':'id', 'month_':'month', 'year_':'year'}, inplace=True)

base_p.head(10)

,id,year,month,epoc_0,epoc_1,epoc_2,epoc_3,epoc_4
0,524006,2015,9,0.0,0.0,0.0,0.0,1.0
1,524006,2016,10,0.0,0.0,0.0,1.0,0.0
2,524006,2017,1,0.0,1.0,0.0,0.0,0.0
3,524006,2017,5,0.0,1.0,0.0,0.0,0.0
4,524006,2017,8,0.0,1.0,0.0,0.0,0.0
5,540333,2015,12,0.0,0.0,1.0,0.0,0.0
6,604083,2015,9,0.0,0.0,0.0,0.0,1.0
7,622031,2016,3,0.0,0.0,1.0,0.0,0.0
8,622031,2016,5,0.0,0.0,1.0,0.0,0.0
9,622031,2017,5,0.0,0.0,1.0,0.0,0.0


## 3. Merge with ids table

In [32]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_epoc = ids_mensual.merge( base_p, how='left')
base_final_epoc.head()

,id,year,month,epoc_0,epoc_1,epoc_2,epoc_3,epoc_4
0,500547,2016,1,NaN,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN,NaN


In [33]:
base_final_epoc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53280 entries, 0 to 53279
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      53280 non-null  int64  
 1   year    53280 non-null  int64  
 2   month   53280 non-null  int64  
 3   epoc_0  125 non-null    float64
 4   epoc_1  125 non-null    float64
 5   epoc_2  125 non-null    float64
 6   epoc_3  125 non-null    float64
 7   epoc_4  125 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 3.7 MB
